In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [2]:
house_all = pd.read_csv("./houseMaster.csv", index_col=0)

/Users/wang/anaconda3/envs/geopandas-test/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
house_all.head()

,Unnamed: 0.1,Unnamed: 0.1.1,house_id,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,...,taxi_pu_time_sec_sum,uber_trips,taxi_do_v_median,taxi_do_time_sec_median,taxi_do_distance_median,taxi_do_passenger_sum,taxi_do_passenger_median,taxi_do_time_sec_sum,taxi_do_distance_sum,sicong_cannot_impute
0,0,0,2,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7351,60,,...,41.0,1.0,14.220974,3089.0,44373.72104,2.0,1.0,3738.0,53561.26521,0
1,1,1,3,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,78,,...,41.0,1.0,14.220974,3089.0,44373.72104,2.0,1.0,3738.0,53561.26521,0
2,2,2,4,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,34,,...,41.0,1.0,14.220974,3089.0,44373.72104,2.0,1.0,3738.0,53561.26521,0
3,3,3,6,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7317,126,,...,41.0,1.0,17.428190,3738.0,58606.28901,2.0,1.0,3738.0,117212.57800,0
4,4,4,7,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7316,93,,...,41.0,1.0,17.428190,3738.0,58606.28901,2.0,1.0,3738.0,117212.57800,0


In [85]:
from sklearn.neighbors import RadiusNeighborsRegressor, KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [86]:
# refine dataset
feature_col = [
    'Unnamed: 0.1',
    'Unnamed: 0.1.1',
    'house_id',
    'BOROUGH',
    'NEIGHBORHOOD',
    # 'BUILDING CLASS CATEGORY',
    # 'TAX CLASS AT PRESENT',
    'BLOCK',
    'LOT',
    'EASE-MENT',
    'BUILDING CLASS AT PRESENT',
    'ADDRESS',
    'APARTMENT NUMBER',
    'ZIP CODE',
    'RESIDENTIAL UNITS',
    'COMMERCIAL UNITS',
    'TOTAL UNITS',
    'LAND SQUARE FEET',
    'GROSS SQUARE FEET',
    'YEAR BUILT',
    'TAX CLASS AT TIME OF SALE',
    'BUILDING CLASS AT TIME OF SALE',
    'SALE PRICE',
    'SALE DATE',
    'latitude_x',
    'longitude_x',
    'SubwayDist',
    'ClosestSubway',
    'numSub5min',
    'numSub10min',
    'id',
    'SubwayDistHaversine',
    'ClosestSubwayHaversine',
    'bike_minDist',
    'bike_minStationID',
    'bike_station.id',
    'bike_nRides_start',
    'bike_durationMed_start',
    'bike_startTimeAvg',
    'bike_startTimeSd',
    'bike_endLatAvg',
    'bike_endLatSd',
    'bike_endLongAvg',
    'bike_endLongSd',
    'bike_females_start',
    'bike_ageMed_start',
    'bike_nRides_end',
    'bike_durationMed_end',
    'bike_endTimeAvg',
    'bike_endTimeSd',
    'bike_startLatAvg',
    'bike_startLatSd',
    'bike_startLongAvg',
    'bike_startLongSd',
    'bike_females_end',
    'bike_ageMed_end',
    'tracts',
    'latitude_y',
    'longitude_y',
    'x',
    'y',
    'pickup_missing_originally',
    'pickup_count',
    'pickup_trip_id',
    'pickup_trip_med_dist',
    'pickup_trip_med_time',
    'pickup_trip_med_speed',
    'pickup_trip_avg_passenger',
    'dropoff_missing_originally',
    'dropoff_count',
    'dropoff_trip_id',
    'dropoff_trip_med_dist',
    'dropoff_trip_med_time',
    'dropoff_trip_med_speed',
    'dropoff_trip_avg_passenger',
    'Unnamed: 0_x',
    'taxi_pu_time_sec_median',
    'taxi_pu_v_median',
    'taxi_pu_distance_sum',
    'taxi_pu_passenger_sum',
    'taxi_pu_passenger_median',
    'taxi_pu_distance_median',
    'taxi_pu_time_sec_sum',
    'uber_trips',
    'taxi_do_v_median',
    'taxi_do_time_sec_median',
    'taxi_do_distance_median',
    'taxi_do_passenger_sum',
    'taxi_do_passenger_median',
    'taxi_do_time_sec_sum',
    'taxi_do_distance_sum',
    'sicong_cannot_impute'
]

data = house_all.sample(frac=1)

data = house_all.loc[:, feature_col].copy()
data = house_all.loc[:, ["latitude_x", "longitude_x", "SALE PRICE"]].copy()

In [87]:
# training test split
train_n = len(data) // 10 * 7
val_n = len(data) // 10 * 1
test_n = len(data) - train_n - val_n

train = data.iloc[:train_n, :]
val = data.iloc[train_n:(train_n+val_n), :]
test = data.iloc[(train_n+val_n):, :]

In [ ]:
lat_mean = train.latitude_x.mean()
lat_sd = train.latitude_x.std()
long_mean = train.longitude_x.mean()
long_sd = train.longitude_x.std()

train

In [87]:
train_x = 100 * np.array(train.loc[:, ["latitude_x", "longitude_x"]])
train_y = np.array(train.loc[:, "SALE PRICE"])
val_x = 100 * np.array(val.loc[:, ["latitude_x", "longitude_x"]])
val_y = np.array(val.loc[:, "SALE PRICE"])
test_x = 100 * np.array(test.loc[:, ["latitude_x", "longitude_x"]])
test_y = np.array(test.loc[:, "SALE PRICE"])

In [88]:
RNR = RadiusNeighborsRegressor(radius=200, weights="distance", algorithm="auto", p=1).fit(X=train_x, y=train_y)
val_pre = RNR.predict(X=val_x)
#val_mse = mean_squared_error(val_y, val_pre)

/Users/wang/anaconda3/envs/geopandas-test/lib/python3.7/site-packages/sklearn/neighbors/regression.py:327: UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
  warnings.warn(empty_warning_msg)


In [98]:
KNR = KNeighborsRegressor(n_neighbors=80, weights="distance", algorithm="auto", p=1).fit(X=train_x, y=train_y)
val_pre = KNR.predict(X=val_x)
val_mse = mean_squared_error(val_y, val_pre)

In [99]:
val_mse

454267637381.83923

In [71]:
data.head()

,latitude_x,longitude_x,SALE PRICE
0,40.530317,-74.219194,460000.0
1,40.532668,-74.222289,550000.0
2,40.532515,-74.221103,450000.0
3,40.535582,-74.221539,648000.0
4,40.535688,-74.222679,509000.0
